# Previsão de Mortes por COVID

O projeto consistiu em produzir módulos, scripts, notebooks para as tarefas de análise exploratória e modelagem preditiva do número de mortes pela COVID em 2020.

Os dados oferecidos são acessados pela api:

* COVID TRACKING: https://api.covidtracking.com 
    - Valores históricos: /v1/us/daily.csv
    - Meta dados dos Estados: /v1/states/info.csv
    - Valores históricos para Estados: /v1/states/{state}/current.csv
   
Além desses, foram também consultados:

* POPULATION ESTIMATES API: https://api.census.gov/data/2019/pep/population
    - Consulta a estimativas populacionais nos EUA.

### Ferramentas desenvolvidas

 * `api_manager`módulo
 
     Esse módulo oferece a class `ApiManager` que automatiza os requests das APIs, o gerencimento dos diretório onde os dados serão armazenados ao mesmo tempo que ofere rápido acesso a eles.

* `DataframeTransformer` (módulo transformers.py)

    Este módulo perimte construir pipelines de transformação em estruturas de dados. Nesse projeto ele foi utilizado para fazer pipelines de processamento dos dados buscados na API.

    Foi pensado em ser versátil e prático e por isso ele é capaz de utilizar com facilidade funções customizadas para integrar o pipeline.

  ### Scripts & Resources
  
        .resources/
            |__ 
  
       .scripts/
            |__retrieve-data-nb01.py        # coletar dados do covid
            |__retrieve-data-by-states.py   # coletar dados por estados
            |__retrieve-data-census.py      # coletar dados demográficos
            |__processing_data_by_states.py # preparar dados para EDA
            |__make_datasets.py             # construir `traininig` e `test` sets
            |__training.py                  # trainer modelo de regressão
            |__model_evaluation.py          # Calcular previsão e métrica de performance para o `test_set`

## Notebooks
---

Os notebooks se encontram na raiz do repositório e foram utilizados para *análise exploratória* (EDA), vizualização e avaliação do modelo.

A Análise Exploratória dos dados nesse projeto teve como foco primário estudar as variáveis acessíveis na COVID TRACKING API e definir estratégias para modelagem preditiva. Nesse escopo encontram-se atividades de visualização, tratamento, engenharia de atributos, análises estatísticas e de mineraçao.

* `01_exploração_inicial.ipynb`
    
    Esse notebook investiga as características da séries temporais das variáveis relacionadas a epidemia do covid. Define a melhor variável alvo com seus possíveis preditores, além de investigar tratamentos nos dados visando ganhos no modelo preditivo.

* `02_EDA_avancada.ipynb`

    O foco deste notebook é visualização e mineração para definição de estratégias para o modelo preditivo. Foram empegadas técnicas e/ou algorítimos para decomposição espectral das séries temporais e clusterização para detecção de anomalias. A mineração se voltou para com essas técnicas buscar perfis regionais de evolução da covid no território americano.

* `03_Avaliacao_regressao,ipynb`

    Este é um notebook dedicado para visualização dos resultados/previsões do modelo nos dados de teste.

    Aqui foram  exploradas tanto as taxas de mortes diárias como acumuladas ao longo do período de teste.

# EDA – Análise Exploratória
---

**OBJETIVOS**

* Identificar ou criar variáveis alvos que mais se adequem ao objetivo do projeto.
    > Nesse projeto as variáveis representavam acúmulos enquanto outras taxas de variação. Estas últimas foram o foco da análise visando sua utilidade preditiva.
    
* Minerar dados e insights no desenho do modelo preditivo mais adequado
    > Com o foco sendo prever mortes no ambito nacional, como o ambiente local (estados) importam?

* Desenvolver técnicas de abordagem que serão proveitosas nas etapas posteriores
    > Algumas das dificuldades aparecem em virtudes das primeiras escolhas no projeto, como o descarte de dados, ou o caso da variável algo escolhida `deathIncrease` ter oscilações
    
* Tratamento de NULLs

### EDA – Parte 1
---
* Algumas variáveis apresentam características semelhantes às da alvo.
* Variável alvo tem características oscilatórias.
    
   
<center><img src="data/imgs/deathIncrease_noisy.png" width="700"></center>

### EDA – Parte 1
---
 * *Utilizando do modelo Seasonal-Trend Decomposition (LOESS)*
     A modelagem da sasonalidade por STL foi feita com a minimização e diagnóstico dos resíduos.
     
<img src="data/imgs/stl_trends_deaths.png" width="700">

### EDA – Parte 1
---

* Capacidade preditiva foi avaliada pela quantidade de *lags* que levam à maximização da correlação (Pearson) entre as séries temporais

    


        [inIcuCurrently...........] Linear Corr (0.820) maximizado para lag => 00
        [hospitalizedCurrently....] Linear Corr (0.616) maximizado para lag => 11
        [hospitalizedIncrease.....] Linear Corr (0.808) maximizado para lag => 10
        [positiveIncrease.........] Linear Corr (0.401) maximizado para lag => 31
   

### EDA – Parte 2
---

* O objetivo nessa segunda fase é minerar dados e informações para a modelagem preditiva.
* A análise ganha foco na variável de interesse e o volume de dados é aumentado.
* A análise espectral e detecção de anomalias por clusterização são impregadas nesse intuito.

> Como essa parte da EDA trata com dados de estados, a variável `deathIncrease` foi alterado para representar a *taxa de mortes por covid para cada 100.000 pessoas".

### EDA – Parte 2
---

<center><img src="data/imgs/components.png" width="700"></center>

* As componentes representam características em comum compartilhadas entre os estados.
    > A segunda componente indica um crescimento da taxa de mortes próximo ao final do ano.
 
* A varância explicada mostra como poucas carcterísticas (4 componentes) contam com ~95% da varância de um conjunto de dados muito maior.
    > Nesse caso, elas indicam padrões de evolução da pandemia em certos locais. Estados com a 2a. compnente principal destacada são aqueles que tiveram uma aceleração na taxa de mortes.

### EDA – Parte 2
---

<center><img src="data/imgs/projeçoes_estados.png" width="800"></center>

* A segunda componente, que indicava um aumento expresisvo na taxa de mortes por covid, foi analisada nos estados em que ela teve maior destaque.
    Esses estados, que foram vistos na imagem, concentram uma população muito restrita em relação ao nacional (menos de 5%). Assim essa anomalia não vai influir perigosamente no modelo.

### EDA – Parte 2
---
**BOTTOMLINE**

O comportamento do covid nos eua evoluiu para um estado em que ela é similar mesmo em regiões geograficamente afastadas. 

O período inicial dos dados será descartado pois nesse período os estados apresentam séries bastantes dispersas. A partir destas informações já é possível desenhar modelos preditivos e de *forecasting* basedos nestes dados. Opções são utilizar a própria autocorrelação das séries nacionais em modelos autoregressivos como ARIMA ou utilizar utilizar preditores.


# Previsão de Mortes por COVID
---

* 
O modelo se aproxima bastante da taxa de mortes suavizada com Loess e se faz assim representativo da série temporal dos registros de `deathIncrease`. Ao lado, ao integrar o modelo, o o viés (bias) do model vai ficando aparente. 


<center><img src="data/imgs/model_performance.png"></center>